
### Objective: 

In this assignment, implement the Node2Vec algorithm, a random-walk-based GNN, to learn node embeddings. Train a classifier using the learned embeddings to predict node labels.

### Dataset: 

Cora dataset: The dataset consists of 2,708 nodes (scientific publications) with 5,429 edges (citations between publications). Each node has a feature vector of size 1,433, and there are 7 classes (research topics).
Skeleton Code:

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
from node2vec import Node2Vec  # Importing Node2Vec for the random walk

# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Prepare data
data = dataset[0]

# Convert to networkx for random walk
import networkx as nx
G = to_networkx(data, to_undirected=True)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=8) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")


Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 8): 100%|██████████| 25/25 [00:07<00:00,  3.16it/s]
/tmp/ipykernel_160/2407542284.py:45: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))


Epoch 0, Loss: 1.9151047468185425
Epoch 10, Loss: 1.2516547441482544
Epoch 20, Loss: 0.9109947085380554
Epoch 30, Loss: 0.757879912853241
Epoch 40, Loss: 0.6823053359985352
Epoch 50, Loss: 0.6378163695335388
Epoch 60, Loss: 0.6071047782897949
Epoch 70, Loss: 0.5842233896255493
Epoch 80, Loss: 0.5664762258529663
Epoch 90, Loss: 0.5522255301475525
Training complete!


## Explanation:
Node2Vec generates node embeddings by simulating random walks on the graph. These walks capture structural properties of nodes.
The generated embeddings are then used to train a classifier for predicting node labels.
The Cora dataset is a benchmark graph where nodes are papers and edges are citations.

## Questions (1 point each):
1. What would happen if we increased the number of walks (num_walks) per node? How might this affect the learned embeddings?
2. What would happen if we reduced the walk length (walk_length)? How would this influence the structural information captured by the embeddings?

4. What would happen if we used directed edges instead of undirected edges for the random walks?
5. What would happen if we added more features to the nodes (e.g., 2000-dimensional features instead of 1433)?
6. What would happen if we used a different dataset with more classes? Would the classifier performance change significantly?
8. What would happen if we used a larger embedding dimension (e.g., 128 instead of 64)? How would this affect the model’s performance and training time?



### Extra credit: 
1. What would happen if we increased the window size (window) for the skip-gram model? How would it affect the embedding quality?

## No points, just for you to think about
7. What would happen if we removed self-loops from the graph before training Node2Vec?

9. What would happen if we applied normalization to the node embeddings before feeding them to the classifier?

What would happen if we increased the number of walks (num_walks) per node? How might this affect the learned embeddings?

My intuition is that increasing the number of walks would increase the amount of time to generate the embeddings. This is because the algorithm will increase the amount of walks it starts from each node. The embeddings will be more comprehensive though because the algorithm will capture more of the neighborhood of the root node. It can better generate vectors because it will have a more thorough view of a node's neighbors.

I think that increasing the number of walks would increase model performance because of the improvement of the embeddings.

In [2]:
G = to_networkx(data, to_undirected=True)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=500, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 250/250 [01:23<00:00,  2.99it/s]


Epoch 0, Loss: 1.9552465677261353
Epoch 10, Loss: 1.2400709390640259
Epoch 20, Loss: 0.9010287523269653
Epoch 30, Loss: 0.7543397545814514
Epoch 40, Loss: 0.6809619069099426
Epoch 50, Loss: 0.6386410593986511
Epoch 60, Loss: 0.6100683808326721
Epoch 70, Loss: 0.58897465467453
Epoch 80, Loss: 0.5725937485694885
Epoch 90, Loss: 0.5593566298484802
Training complete!


In [3]:
(0.5448827743530273 - 0.557102620601654) / 0.557102620601654

-0.02193464147669893

We see a 2% decrease in loss by increasing the num_walks from 200 to 500. I think a strong consideration for this parameter is balancing computation time and performance. It will be important to figure out when the increase in computation time outweights the small increase in performance.

What would happen if we reduced the walk length (walk_length)? How would this influence the structural information captured by the embeddings?

If we reduced the walk length, the structural information about the graph will be less. This is because the random walks won't go as deep into the graph so we will miss the information that is not in the immediate neighborhood of the root node. I think that doing this would reduce the performance of the overall model.

In [4]:
G = to_networkx(data, to_undirected=True)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=15, num_walks=200, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 100/100 [00:16<00:00,  6.11it/s]


Epoch 0, Loss: 2.036820411682129
Epoch 10, Loss: 1.2899699211120605
Epoch 20, Loss: 0.9394242763519287
Epoch 30, Loss: 0.7780355215072632
Epoch 40, Loss: 0.6983869075775146
Epoch 50, Loss: 0.6521328091621399
Epoch 60, Loss: 0.620934247970581
Epoch 70, Loss: 0.5978969931602478
Epoch 80, Loss: 0.5800913572311401
Epoch 90, Loss: 0.5658071041107178
Training complete!


In [5]:
(0.5534951090812683 - 0.557102620601654) / 0.557102620601654

-0.006475488333710831

We see a very slight decrease in loss by changing the walk length from 30 to 15. This is not what I expected but I guess in the context of the problem it makes sense. A paper's classification could be very dependent on its immediate neighborhood so changing the walk length is just a matter of extra noise.

What would happen if we used directed edges instead of undirected edges for the random walks?

I think that we would lose a lot of the information in the graph because all the edges are now directional. A lot of edges that existed in the undirected graph are no more in the directed graph. Based on the nature of this problem, I think that having the bi-directional edges as in the undirected graphs is worthwhile. So I think that there would be a decrease in performance.

In [6]:
G = to_networkx(data, to_undirected=False)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")


Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 100/100 [00:32<00:00,  3.10it/s]


Epoch 0, Loss: 1.9904170036315918
Epoch 10, Loss: 1.2779759168624878
Epoch 20, Loss: 0.9268447160720825
Epoch 30, Loss: 0.770454466342926
Epoch 40, Loss: 0.6934294700622559
Epoch 50, Loss: 0.648582398891449
Epoch 60, Loss: 0.6187095046043396
Epoch 70, Loss: 0.5969102382659912
Epoch 80, Loss: 0.580097496509552
Epoch 90, Loss: 0.5665943622589111
Training complete!


In [9]:
(0.5665943622589111 - 0.557102620601654) / 0.557102620601654

0.01703768983711885

What would happen if we added more features to the nodes (e.g., 2000-dimensional features instead of 1433)?

I'd assume that we'd see an increase in performance if we added more features. More features would mean more data for the embedding and modeling algorithm to learn from. It could make connections in the new data that could reveal better classifications. However, we would also tradeoff training time for both the embedding and modeling steps. There would be more computations that the algorithms would need to make, which would inherently slow down the whole process.

What would happen if we used a different dataset with more classes? Would the classifier performance change significantly?

I don't think that the classifier performance will change significantly. The nodes are mapped into a space where the more similar nodes have a more similar embedding. These embeddings won't change if there are more classes, so I think that the model will still be able to use them to differentiate between the extra classes. In other words, it should still map the more similar nodes to the same class most of the time, regardless of how many classes there are.

What would happen if we used a larger embedding dimension (e.g., 128 instead of 64)? How would this affect the model’s performance and training time?

In [8]:
G = to_networkx(data, to_undirected=True)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=128, walk_length=30, num_walks=200, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(128, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    classifier.train()
    optimizer.zero_grad()
    
    # Get node embeddings as input
    output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
    
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")


Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 100/100 [00:32<00:00,  3.08it/s]


Epoch 0, Loss: 2.007720708847046
Epoch 10, Loss: 1.1136757135391235
Epoch 20, Loss: 0.7734028100967407
Epoch 30, Loss: 0.6548328399658203
Epoch 40, Loss: 0.5966346263885498
Epoch 50, Loss: 0.5577636361122131
Epoch 60, Loss: 0.5290225148200989
Epoch 70, Loss: 0.5069485902786255
Epoch 80, Loss: 0.48921290040016174
Epoch 90, Loss: 0.4743979275226593
Training complete!


In [10]:
(0.4743979275226593 - 0.557102620601654) / 0.557102620601654

-0.1484550422499829

By increasing the embedding dimension from 64 to 128, we see in 14% decrease in loss. However, the time to train was much longer. This is probably because we naturally have more numbers we need to work and hence more computations, which add extra time in computation. 